In [1]:
import uproot
import awkward as ak
import os
import succolib as sl
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from random import randint
from matplotlib.colors import LogNorm

In [2]:
# file names and path
# ---
# open test data with:
# treename = "t"
# filepath = "data_test/"
# filenames = [
#     "run" + "650272" + "_00000*.root",
#     "run" + "650272" + "_00001*.root",
# ]
# ---
treename = "t"
filepath = "data_test/"
filenames = [
    "run" + "650272" + "_00000*.root",
    "run" + "650272" + "_00001*.root",
]

# (max.) nr. of waveforms to open
nwf0 = 1e10

# waveform channel map
ind_wave = {
    "Cal" : 1,
}

# fast analysis channel map relative to waveform channel map
ind_ph = ind_wave

# waveform info
# ---
# > polPos - if False, reverse waveform polarity
# > nBinsBase - nr. of ticks at the beginning for baseline computation
# > bDigiTimeCalib - calibrate the time ticks? ticks --> ns
# > digiTimeCalib - in ns/ticks
# > bDigiSignalCalib - calibrate the signal amplitude? ADC --> mV
# > digiSignalCalib - in mV/ADC
# > upperLimWfVis - cut waveform visualisation with amplitude above this limit
# ---
wf_info = {
    "Cal" : {
        "polPos" : True, 
        "nBinsBase" : 50, 
        "bDigiTimeCalib" : True, 
        "digiTimeCalib" : (1/5e9)*1e9, 
        "bDigiSignalCalib" : True, 
        "digiSignalCalib" : (1/4096)*1000, 
        "upperLimWfVis" : 900, 
    }
}

# tracking info
bTracking = True  # if False, all tracking analysis is excluded
trackingID = ((0, 2), (1, 3))  # IDs of the tracking modules to be used for tracking
dzT1T0 = 72.6  # distance between tracking planes
dzCalT1 = 89.9+8.5  # distance between downstream plane and detector under study
thxShift = 0.003993  # alignment of angular distribution, x
thyShift = 0.004169  # alignment of angular distribution, y

# distributions settings
rterm = 50  # in ohm
pllims = (-10, 220, -20, 1100)  # (t0 [ns], t1 [ns], V0 [mV], V1 [mV])
qtpllims = (0, 3000) # (Q0 [pC], Q1 [pC])
timecut_sig = (28, 64)  # in ns
phcut_sig = (-20000, 20000)  # in mV
timecut_bkg = (150, 200)  # in ns
phcut_bkg = (-20000, 20000)  # in mV

# save figures?
bSaveFigs = True

# list of variables to open
# note: tracking data should be called "xRaw"
variables = [
    "xRaw",
    "wave"+str(ind_wave["Cal"]), 
]

# chosen detector to analyse
var0 = "Cal"

In [ ]:
%%time

# open data
for i, data0 in enumerate(uproot.iterate(
    {filepath+filename : treename for filename in filenames},
    expressions=variables, step_size=100
)):
    if i==0:
        data = data0
    else:
        data = ak.concatenate((data, data0))
        
    if len(data)>nwf0:
        data = data[:nwf0]
        break
        
# create output directory (named after the first of the opened files)
for i, filename in enumerate(filenames):
    if i==0:
        filename0 = filename.split(".")[0].split("_")[0].replace("*", "AST")
        out_path = "./output/%s" % filename0
        if not (os.path.exists(out_path)):
            os.makedirs(out_path)
            print("also created folder: %s" % out_path)
        else:
            print("folder already exists: %s" % out_path)    
    
# remove broken silicon data
if bTracking:
    string0 = "events before removing broken silicon data:\t%d" % len(data)
    print(string0)

    data["bSiliFilter"] = [
        True if len(data["xRaw"][i][abs(data["xRaw"][i])<10])==4 else False for i in range(len(data["xRaw"]))
    ]
    data = data[data["bSiliFilter"]]

    string1 = "events after removing broken silicon data:\t%d" % len(data)
    print(string1)
    
else:
    string0 = "events: %d" % len(data)
    print(string0)

# if plotting the histograms, also write count file
if bSaveFigs:
    if os.path.exists("%s/info_counts.txt" % out_path):
        os.remove("%s/info_counts.txt" % out_path)
    outinfo = open("%s/info_counts.txt" % out_path, "w")
    outinfo.write(string0+"\n")
    if bTracking:
        outinfo.write(string1+"\n")
    outinfo.close()
    
    
    


print("---")

folder already exists: ./output/run650272
events before removing broken silicon data:	1788


In [ ]:
# add tracking

if bTracking:
    data["thInRaw0"] = sl.zAngle(data["xRaw"][:, trackingID[0][1]], dzT1T0, data["xRaw"][:, trackingID[0][0]], 0)
    data["thInRaw1"] = sl.zAngle(data["xRaw"][:, trackingID[1][1]], dzT1T0, data["xRaw"][:, trackingID[1][0]], 0)
    data["thIn0"] = sl.zAngle(data["xRaw"][:, trackingID[0][1]], dzT1T0, data["xRaw"][:, trackingID[0][0]], 0) - thxShift
    data["thIn1"] = sl.zAngle(data["xRaw"][:, trackingID[1][1]], dzT1T0, data["xRaw"][:, trackingID[1][0]], 0) - thyShift
    data["xCal0"] = sl.zProj(data["xRaw"][:, trackingID[0][1]], dzT1T0, data["xRaw"][:, trackingID[0][0]], 0, dzCalT1)
    data["xCal1"] = sl.zProj(data["xRaw"][:, trackingID[1][1]], dzT1T0, data["xRaw"][:, trackingID[1][0]], 0, dzCalT1)

In [ ]:
x_wf = {}

for var in [var for var in variables if "wave" in var]:
    vartemp = {v: k for k, v in ind_wave.items()}[int(var.replace("wave", ""))]
    
    # wf, convert to physical quantities
    x_wf[vartemp] = np.arange(
        0,
        len(data[var][0])*(wf_info[vartemp]["digiTimeCalib"] if wf_info[vartemp]["bDigiSignalCalib"] else 1),
        (wf_info[vartemp]["digiTimeCalib"] if wf_info[vartemp]["bDigiTimeCalib"] else 1)
    )
    data[var] = data[var]*(wf_info[vartemp]["digiSignalCalib"] if wf_info[vartemp]["bDigiSignalCalib"] else 1)
    
    # wf, change polarity if negative
    if not wf_info[vartemp]["polPos"]:
        data[var] = -data[var]
    
    # wf, compute and subtract baseline
    ind_wave[vartemp+"_Base"] = 1000 + ind_wave[vartemp]
    ind_wave[vartemp+"_BaseSub"] = 2000 + ind_wave[vartemp]
    data["wave"+str(ind_wave[vartemp+"_Base"])] = np.mean(data[var][:, 0: wf_info[vartemp]["nBinsBase"]])
    data["wave"+str(ind_wave[vartemp+"_BaseSub"])] = data[var] - data["wave"+str(ind_wave[vartemp+"_Base"])]

In [ ]:
# beam angles and profile at chosen detector

fidcut = ((1.5, 5.5), (2.5, 6.5))  # fiducial cut - ((x0, x1), (y0, y1))

if True & bTracking:
    xplot = np.linspace(-0.02, 0.02, 50)  # binning for angle plots
    xplot2 = np.linspace(-1, 11, 100)  # binning for profile and spot plots
    sigCutSpot = 200  # minimum signal size (rough estimate) for efficiency mapping

    fig = plt.figure(figsize=(10, 9))
    
    for i in range(2):
        
        # unaligned angles
        ax = fig.add_subplot(321+i)
        ax.grid(True)
        ax.set_title("unaligned ang., dir. %d"%i)
        hist = plt.hist(data["thInRaw%d"%i], bins=xplot, histtype="step") ;
        xfit, yfit = hist[1][:-1]+0.5*(hist[1][1]-hist[1][0]), hist[0]
        par, _ = curve_fit(sl.fGaus, xfit, yfit, p0=(max(yfit), xfit[yfit==max(yfit)][0], 0.1))
        plt.plot(xplot, sl.fGaus(xplot, *par))
        plt.axvline(0, c="k", ls=":")
        print("before centering: center along %d is @ %f" % (i, par[1]))

        # aligned angles
        ax = fig.add_subplot(323+i)
        ax.grid(True)
        ax.set_title("aligned ang., dir. %d"%i)
        hist = plt.hist(data["thIn%d"%i], bins=xplot, histtype="step") ;
        xfit, yfit = hist[1][:-1]+0.5*(hist[1][1]-hist[1][0]), hist[0]
        par, _ = curve_fit(sl.fGaus, xfit, yfit, p0=(max(yfit), xfit[yfit==max(yfit)][0], 0.1))
        plt.plot(xplot, sl.fGaus(xplot, *par))
        plt.axvline(0, c="k", ls=":")
        print("after centering: center along %d is @ %f" % (i, par[1]))

        # aligned beam profiles at chosen detector
        ax = fig.add_subplot(325+i)
        ax.grid(True)
        ax.set_title("profile at detector, dir. %d"%i)
        plt.hist(data["x%s%d" % (var0, i)], bins=xplot2, histtype="step") ;
    
    fig.tight_layout()
    fig.show()
    
    if bSaveFigs:
        fig.savefig("%s/beam.jpg" % out_path, dpi=200)
        
    # 2d-histogram with beam spot
    fig2 = plt.figure(figsize=(10, 5))
    sel_num = (np.amax(data["wave"+str(ind_wave[var0])], axis=1)>sigCutSpot)
    num = (np.array(data["x%s0" % var0])[sel_num], np.array(data["x%s1" % var0])[sel_num])
    den = (np.array(data["x%s0" % var0]), np.array(data["x%s1" % var0]))
    
    ax2 = fig2.add_subplot(121)
    ax2.set_title("whole beam spot")
    ax2.hist2d(den[0], den[1], bins=(xplot2, xplot2), cmap="viridis")
    ax2.grid(True)
    
    ax2 = fig2.add_subplot(122)
    ax2.set_title("detector efficiency spot, signal > %d" % sigCutSpot)
    sl.hist2dRatio(num[0], num[1], den[0], den[1], bins=(xplot2, xplot2), ax=ax2, cmap="viridis") ;
    ax2.plot(
        (fidcut[0][0], fidcut[0][1], fidcut[0][1], fidcut[0][0], fidcut[0][0]),
        (fidcut[1][0], fidcut[1][0], fidcut[1][1], fidcut[1][1], fidcut[1][0]),
        color="red", ls="--"
    )
    ax2.grid(True)
    
    fig2.tight_layout()
    fig2.show()
    
    if bSaveFigs:
        fig2.savefig("%s/eff_map.jpg" % out_path, dpi=200)

In [ ]:
# apply fiducial cut

if bTracking:
    data["bFidBox"] = [
        (
            (data["x%s0" % var0][i]>fidcut[0][0]) & (data["x%s0" % var0][i]<fidcut[0][1]) &\
            (data["x%s1" % var0][i]>fidcut[1][0]) & (data["x%s1" % var0][i]<fidcut[1][1])
        ) for i in range(len(data["xRaw"]))
    ]
    data = data[data["bFidBox"]]

    string2 = "events after applying fiducial box:\t%d" % len(data)
    print(string2)

    # if plotting the histograms, also write count file
    if bSaveFigs:
        outinfo = open("%s/info_counts.txt" % out_path, "a")
        outinfo.write(string2+"\n")
        outinfo.close()

In [ ]:
%%time

# single-channel waveform processing

brandpeak = True  # if True (False) a random one among the (the minimum of the) identical global maxima is selected 

bplwf = True  # plot the waveforms?

if bplwf:
    bplsingle = lambda i, wf : (True if (i%1==0) else False) & (max(wf)<wf_info[var0]["upperLimWfVis"])
    
ph = []
tp = []
bs = []
qt = []

if bplwf:
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.grid(True)

for i, y0 in enumerate(data["wave"+str(ind_wave[var0])]):
    var = var0 + "_BaseSub"
    y = data["wave"+str(ind_wave[var])][i]
    x = x_wf[var0]
    base = np.mean(data["wave"+str(ind_wave[var0+"_Base"])])
    
    ph.append(np.max(y))
    xmaxs = x[y==np.max(y)]
    tp.append(xmaxs[randint(0, len(xmaxs)-1)] if brandpeak else np.min(xmaxs))
    bs.append(base)
    qt.append(((x[1]-x[0])/rterm)*sum(y))
    
    if (bplwf & bplsingle(i, y)):
        ax.plot(x[0: wf_info[var0]["nBinsBase"]], y0[0: wf_info[var0]["nBinsBase"]], color="red", lw=0.1)
        ax.plot(x, y, color="C0", lw=0.1)
        
if bplwf:
    ax.axvline(0, color="k", lw=1, ls=":")
    ax.axvline(x[wf_info[var0]["nBinsBase"]], color="red", lw=1, label="cut for baseline calculation")
    ax.set_xlabel("time [%s]" % ("ns" if wf_info[vartemp]["bDigiTimeCalib"] else "ticks"))
    ax.set_ylabel("voltage [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    ax.set_xlim((pllims[0], pllims[1]))
    ax.set_ylim((pllims[2], pllims[3]))
    ax.legend()
    fig.tight_layout()
    
    if bSaveFigs:
        fig.savefig("%s/wfs_single.jpg" % out_path, dpi=200)
    
ph = np.array(ph)
tp = np.array(tp)
bs = np.array(bs)
qt = np.array(qt)

sel_sig = (tp>timecut_sig[0]) & (tp<timecut_sig[1]) & (ph>phcut_sig[0]) & (ph<phcut_sig[1])
sel_bkg = (tp>timecut_bkg[0]) & (tp<timecut_bkg[1]) & (ph>phcut_bkg[0]) & (ph<phcut_bkg[1])

In [ ]:
%%time

# single-channel waveform processing, simply for display in histogram form

if True:  # plot the waveforms in 2d-histogram form?
    
    descFact = 2  # rebinning factor
    
    xfl = np.array([])
    yfl = np.array([])
    
    fig, ax = plt.subplots(figsize=(8, 6))

    for i, y in enumerate(data["wave"+str(ind_wave[var0+"_BaseSub"])]):
        x = x_wf[var0]
        xfl = np.concatenate((xfl, np.array(x[::descFact])))
        yfl = np.concatenate((yfl, np.array(y[::descFact])))

    ax.axvline(0, color="k", lw=1, ls=":")
    ax.axvline(x[wf_info[var0]["nBinsBase"]], color="red", lw=1, label="cut for baseline calculation")
    bins = (np.arange(pllims[0], pllims[1], (pllims[1]-pllims[0])/400*descFact), 400)
    hist = ax.hist2d(xfl, yfl, bins=bins, cmin=1, cmap="jet", norm=LogNorm())
    fig.colorbar(hist[3], ax=ax)
    ax.grid(True)
    ax.set_xlim((pllims[0], pllims[1]))
    ax.set_ylim((pllims[2], pllims[3]))
    ax.set_xlabel("time [%s]" % ("ns" if wf_info[vartemp]["bDigiTimeCalib"] else "ticks"))
    ax.set_ylabel("voltage [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    ax.legend()
    fig.tight_layout()
    
    if bSaveFigs:
        fig.savefig("%s/wfs_hist2d.jpg" % out_path, dpi=200)

In [ ]:
def drawbox(limsx, limsy):
    box = (
        (limsx[0], limsx[0], limsx[1], limsx[1], limsx[0]),
        (limsy[0], limsy[1], limsy[1], limsy[0], limsy[0])
    )
    return box[0], box[1]

nbins_ph = 150  # nr. of bins in PH plots
nbins_tp = 100  # nr. of bins in time peak plots
nbins_qt = 200  # nr. of bins in charge plots
phSigmaToMeanMult = 1  # for PH fit: factor to calculate width initial parameter from the mean one
qtSigmaToMeanMult = 2  # for charge fit: factor to calculate width initial parameter from the mean one

In [ ]:
# aggregate distributions

if True:

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2)

    ax = axs[0, 0]
    ax.hist2d(ph, tp, bins=(np.linspace(pllims[2], pllims[3], nbins_ph), np.linspace(pllims[0], pllims[1], nbins_tp)), norm=LogNorm());
    ax.plot(drawbox(phcut_bkg, timecut_bkg)[0], drawbox(phcut_bkg, timecut_bkg)[1], label="pedestal (sel . box)", color="purple", lw=1)
    ax.plot(drawbox(phcut_sig, timecut_sig)[0], drawbox(phcut_sig, timecut_sig)[1], label="signal (sel . box)", color="C1", lw=1)
    ax.axvline(0, color="k", lw=1, ls=":")
    ax.axhline(0, color="k", lw=1, ls=":")
    ax.axhline(x[wf_info[var0]["nBinsBase"]], color="red", lw=1, label="cut for baseline calculation")
    ax.set_ylim((pllims[0], pllims[1]))
    ax.set_xlim((pllims[2], pllims[3]))
    ax.grid(True)
    ax.set_ylabel("time [%s]" % ("ns" if wf_info[vartemp]["bDigiTimeCalib"] else "ticks"))
    ax.set_xlabel("PH [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    ax.legend()
    
    ax = axs[0, 1]
    ax.hist(tp, bins=np.linspace(pllims[0], pllims[1], nbins_tp), alpha=0.2, color="C0", orientation="horizontal");
    ax.hist(tp[sel_bkg], bins=np.linspace(pllims[0], pllims[1], nbins_tp), histtype="step", color="purple", orientation="horizontal");
    ax.hist(tp[sel_sig], bins=np.linspace(pllims[0], pllims[1], nbins_tp), histtype="step", color="C1", orientation="horizontal");
    ax.axhline(0, color="k", lw=1, ls=":")
    ax.axhline(x[wf_info[var0]["nBinsBase"]], color="red", lw=1, label="cut for baseline calculation")
    ax.set_ylim((pllims[0], pllims[1]))
    ax.set_xscale("log")
    ax.grid(True)
    ax.set_ylabel("time [%s]" % ("ns" if wf_info[vartemp]["bDigiTimeCalib"] else "ticks"))

    ax = axs[1, 0]
    ax.hist(ph, bins=np.linspace(pllims[2], pllims[3], nbins_ph), alpha=0.2, color="C0");
    histpede = ax.hist(ph[sel_bkg], bins=np.linspace(pllims[2], pllims[3], nbins_ph), histtype="step", color="purple");
    pede = histpede[0] * (timecut_sig[1]-timecut_sig[0])/(timecut_bkg[1]-timecut_bkg[0])
    histfit = np.histogram(ph[sel_sig], bins=np.linspace(pllims[2], pllims[3], nbins_ph));
    histfit = (histfit[0] - pede, histfit[1])
    xpl, ypl = histfit[1][:-1]+0.5*(histfit[1][1]-histfit[1][0]), histfit[0]
    ax.step(xpl, ypl, color="C1", where="mid", lw=1)
    ax.axvline(0, color="k", lw=1, ls=":")
    ax.set_xlim((pllims[2], pllims[3]))
    ax.set_yscale("log")
    ax.grid(True)
    ax.set_xlabel("PH [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    
    print("PH spectrum after pedestal subtraction:")
    print(histfit[0])
    
    func = sl.fLandau
    par = (max(ypl), xpl[ypl==max(ypl)][0], xpl[ypl==max(ypl)][0]*phSigmaToMeanMult)
    par, _ = curve_fit(func, xpl, ypl, p0=par)
    xplfit = np.linspace(pllims[2], pllims[3], 1000)
    yplfit = func(xplfit, *par)
    ax.plot(xplfit[yplfit>0.01*np.max(yplfit)], yplfit[yplfit>0.01*np.max(yplfit)], label="landau MPV = %.3f" % par[1], color="green")
    ax.legend()
    
    ax = axs[1, 1]
    ax.hist(qt, bins=np.linspace(qtpllims[0], qtpllims[1], nbins_qt), alpha=0.2, color="C0");
    histpede = ax.hist(qt[sel_bkg], bins=np.linspace(qtpllims[0], qtpllims[1], nbins_qt), histtype="step", color="purple");
    pede = histpede[0] * (timecut_sig[1]-timecut_sig[0])/(timecut_bkg[1]-timecut_bkg[0])
    histfit = np.histogram(qt[sel_sig], bins=np.linspace(qtpllims[0], qtpllims[1], nbins_qt));
    histfit = (histfit[0] - pede, histfit[1])
    xpl, ypl = histfit[1][:-1]+0.5*(histfit[1][1]-histfit[1][0]), histfit[0]
    ax.step(xpl, ypl, color="C1", where="mid", lw=1)
    ax.axvline(0, color="k", lw=1, ls=":")
    #ax.set_xlim((pllims[2], pllims[3]))
    ax.set_yscale("log")
    ax.grid(True)
    ax.set_xlabel("Q [%s]" % ("pC" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC*ticks"))
    
    print("Q spectrum after pedestal subtraction:")
    print(histfit[0])

    func = sl.fLandau
    par = (max(ypl), xpl[ypl==max(ypl)][0], xpl[ypl==max(ypl)][0]*qtSigmaToMeanMult)
    par, _ = curve_fit(func, xpl, ypl, p0=par)
    xplfit = np.linspace(qtpllims[0], qtpllims[1], 1000)
    yplfit = func(xplfit, *par)
    ax.plot(xplfit[yplfit>0.01*np.max(yplfit)], yplfit[yplfit>0.01*np.max(yplfit)], label="landau MPV = %.3f" % par[1], color="green")
    ax.legend()
    
    fig.tight_layout()
    
    if bSaveFigs:
        fig.savefig("%s/distributions.jpg" % out_path, dpi=200)

In [ ]:
# distributions over time

if True:

    blog = False
    nbins_iev = 10

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2)

    xtrend = np.array([i for i in range(len(tp))])

    ax = axs[0, 0]
    ax.set_ylabel("time [%s]" % ("ns" if wf_info[vartemp]["bDigiTimeCalib"] else "ticks"))
    bins = (np.arange(0, len(xtrend), nbins_iev), np.linspace(pllims[0], pllims[1], nbins_tp))
    ytrend = tp
    ax.hist2d(xtrend, ytrend, bins=bins, cmin=1, norm=LogNorm() if blog else None) ;

    ax = axs[0, 1]
    ax.set_ylabel("baseline [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    bins = (np.arange(0, len(xtrend), nbins_iev), 100)
    ytrend = bs
    ax.hist2d(xtrend, ytrend, bins=bins, cmin=1, norm=LogNorm() if blog else None) ;

    ax = axs[1, 0]
    ax.set_ylabel("PH [%s]" % ("mV" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    bins = (np.arange(0, len(xtrend), nbins_iev), np.linspace(pllims[2], pllims[3], nbins_ph))
    ytrend = ph
    ax.hist2d(xtrend, ytrend, bins=bins, cmin=1, norm=LogNorm() if blog else None) ;

    ax = axs[1, 1]
    ax.set_ylabel("Q [%s]" % ("pC" if wf_info[vartemp]["bDigiSignalCalib"] else "ADC"))
    bins = (np.arange(0, len(xtrend), nbins_iev), np.linspace(qtpllims[0], qtpllims[1], nbins_qt))
    ytrend = qt
    ax.hist2d(xtrend, ytrend, bins=bins, cmin=1, norm=LogNorm() if blog else None) ;

    fig.supxlabel("event nr.")
    for axx in axs:
        for ax in axx:
            ax.grid(True)

    fig.tight_layout()
    
    if bSaveFigs:
        fig.savefig("%s/trends_vs_event.jpg" % out_path, dpi=200)